# Interpolation

In [1]:
using Interpolations

include("printmat.jl")   #just a function for prettier matrix printing

printlnPs (generic function with 1 method)

In [2]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))    
else    
    gr(size=(600,400))
end

## Some Data to Be Interpolated

In [3]:
xGrid = collect(linspace(-pi,pi,101))            #equally spaced grid
yGrid = sin.(xGrid)

plot1 = plot(xGrid,yGrid,color=:red,linewidth=2,legend=nothing)
title!("the sin function")
xlabel!("x")
ylabel!("y")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 the sin function 
 
 
 x 
 
 
 y 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 54.7703,196.528 60.1833,207.4 65.5962,218.229 71.0091,228.972 76.422,239.588 81.835,250.033 87.2479,260.267 92.6608,270.25 98.0738,279.942 103.487,289.304 
 108.9,298.301 114.313,306.895 119.725,315.055 125.138,322.746 130.551,329.939 135.964,336.606 141.377,342.72 146.79,348.257 152.203,353.195 157.616,357.515 
 163.029,361.2 168.442,364.234 173.855,366.607 179.268,368.309 184.681,369.332 190.094,369.674 195.506,369.332 200.919,368.309 206.332,366.607 211.745,364.234 
 217.158,361.2 222.571,357.515 227.984,353.195 233.397,348.257 238.81,342.72 244.223,336.606 249.636,329.939 255.049,322.746 260.462,315.055 265.874,306.895 
 271.287,298.301 276.7,289.304 282.113,279.942 287.526,270.25 292.939,260.267 298.352,250.033 303.765,239.588 309.178,228.972 314.591,218.229 320.004,207.4 
 325.417,196.528 330.83,185.656 336.243,174.827 341.655,164.083 347.068,153.468 352.481,143.023 357.894,132.788 363.307,122.806 368.72,113.114 374.133,103.751 
 379.546,94.7549 384.959,86.1602 390.372,78.001 395.785,70.3095 401.198,63.1162 406.611,56.4495 412.023,50.3355 417.436,44.7985 422.849,39.8603 428.262,35.5404 
 433.675,31.8558 439.088,28.8212 444.501,26.4483 449.914,24.7468 455.327,23.7231 460.74,23.3815 466.153,23.7231 471.566,24.7468 476.979,26.4483 482.392,28.8212 
 487.804,31.8558 493.217,35.5404 498.63,39.8603 504.043,44.7985 509.456,50.3355 514.869,56.4495 520.282,63.1162 525.695,70.3095 531.108,78.001 536.521,86.1602 
 541.934,94.7549 547.347,103.751 552.76,113.114 558.173,122.806 563.585,132.788 568.998,143.023 574.411,153.468 579.824,164.083 585.237,174.827 590.65,185.656 
 596.063,196.528 
 "/>

## Preparing the Inter- and Extrapolation

In [4]:
#create interpolation and extrapolation objects (several options are available)
itp = interpolate(yGrid, BSpline(Quadratic(Flat())), OnGrid())
etp = extrapolate(itp, Flat())

#to interpolate the y values at
x              = [0.25;0.75]

2-element Array{Float64,1}:
 0.25
 0.75

## How to Find x (where we want to interpolate) in xGrid (pre-existing grid)

In [5]:
function CompressTo1NPs(x,MinX,MaxX,N)              #x2 = a + b*x, so that x2 is in [1,N]
  b = (N-1)/(MaxX-MinX)
  a = 1 - b*MinX
  x2 = a + b*x
  return x2
end

CompressTo1NPs (generic function with 1 method)

In [6]:
#first find at which indices of xGrid where x would be located 
x_clamped = CompressTo1NPs(x,minimum(xGrid),maximum(xGrid),length(xGrid))   #x_clamped is a linear fn of x

println("We want interpolation of f(x) at x = ")
printmat(x)

println("We can look at the values at x_clamped = ")
printmat(x_clamped)

println("Compare with closest available values in xGrid")
printmat(xGrid[round(Int,x_clamped)])

We want interpolation of f(x) at x = 
     0.250
     0.750

We can look at the values at x_clamped = 
    54.979
    62.937

Compare with closest available values in xGrid
     0.251
     0.754



## Interpolate

In [7]:
#then interpolate as
y_interpolated = itp[x_clamped]

println("x and interpolated values")
printmat([x y_interpolated])

x and interpolated values


## Extrapolate

In [8]:
#extrapolation in- and outside xGrid (...but do really want to do that?) is similar
x2             = [1.25;pi+0.1;pi+0.5]
x2_clamped     = CompressTo1NPs(x2,minimum(xGrid),maximum(xGrid),length(xGrid))
y_extrapolated = etp[x2_clamped]

println("x2 and extrapolated values")
printmat([x2 y_extrapolated])

x2 and extrapolated values
     1.250     0.949
     3.242     0.000
     3.642     0.000



## Looking at the Results

In [9]:
plot2 = plot(xGrid,yGrid,color=:red,linewidth=2,label="sin function")
scatter!(x,y_interpolated,color=:green,markersize=4,label="interpolated")
scatter!(x2,y_extrapolated,color=:blue,markersize=8,label="extrapolated")
title!("the sin function")
xlabel!("x")
ylabel!("y")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 the sin function 
 
 
 x 
 
 
 y 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 52.8153,196.528 57.7008,206.784 62.5863,217 67.4717,227.136 72.3572,237.15 77.2427,247.004 82.1282,256.659 87.0136,266.077 91.8991,275.22 96.7846,284.053 
 101.67,292.54 106.556,300.648 111.441,308.346 116.326,315.602 121.212,322.388 126.097,328.677 130.983,334.445 135.868,339.669 140.754,344.327 145.639,348.403 
 150.525,351.879 155.41,354.742 160.296,356.98 165.181,358.585 170.067,359.551 174.952,359.873 179.838,359.551 184.723,358.585 189.609,356.98 194.494,354.742 
 199.38,351.879 204.265,348.403 209.151,344.327 214.036,339.669 218.921,334.445 223.807,328.677 228.692,322.388 233.578,315.602 238.463,308.346 243.349,300.648 
 248.234,292.54 253.12,284.053 258.005,275.22 262.891,266.077 267.776,256.659 272.662,247.004 277.547,237.15 282.433,227.136 287.318,217 292.204,206.784 
 297.089,196.528 301.975,186.271 306.86,176.055 311.746,165.92 316.631,155.905 321.516,146.051 326.402,136.396 331.287,126.979 336.173,117.835 341.058,109.003 
 345.944,100.516 350.829,92.4074 355.715,84.71 360.6,77.454 365.486,70.6678 370.371,64.3784 375.257,58.6105 380.142,53.3869 385.028,48.7283 389.913,44.6529 
 394.799,41.1769 399.684,38.314 404.57,36.0755 409.455,34.4702 414.341,33.5045 419.226,33.1822 424.111,33.5045 428.997,34.4702 433.882,36.0755 438.768,38.314 
 443.653,41.1769 448.539,44.6529 453.424,48.7283 458.31,53.3869 463.195,58.6105 468.081,64.3784 472.966,70.6678 477.852,77.454 482.737,84.71 487.623,92.4074 
 492.508,100.516 497.394,109.003 502.279,117.835 507.165,126.979 512.05,136.396 516.935,146.051 521.821,155.905 526.706,165.92 531.592,176.055 536.477,186.271 
 541.363,196.528 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 sin function 
 
 
 
 
 
 
 interpolated 
 
 
 
 
 
 
 extrapolated